In [1]:
import random
import numpy as np
import pandas as pd
import requests
import json
from bs4 import BeautifulSoup
import urllib
import time
from urllib.error import HTTPError
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
from PorterStemmer import PorterStemmer

In [36]:
def get_news_text(url):
    try:
        response = urllib.request.urlopen(url)
    except HTTPError:
        return ''
    else:
        html = response.read().decode('utf-8')
        soup = BeautifulSoup(html, features="lxml")
        doc = []
        content = ''
        p_list = soup.find_all('p')
        p_list_new = []
        for i in p_list:
            for text in i.strings:
                doc.append(text)
        content = ''.join(doc)
        return content

In [60]:
# load file
import csv
with open("dataset/MINDlarge_train/behaviors.tsv") as file:
    dataset = []
    for line in file:
        fields = line.strip().split('\t')
        header = ['impression_id','user_id','time','history','impressions']
        d = dict(zip(header, fields))
        dataset.append(d)
dataset_copy = dataset.copy()
dataset_sub = dataset_copy[:1000]
for i in range(len(dataset_sub)):
    dataset_sub[i]['history'] = dataset_sub[i]['history'].split()
    dataset_sub[i]['impressions'] = dataset_sub[i]['impressions'].split()
for i in range(len(dataset_sub)):
    dataset_sub[i]['history'] = [str(x) + '-1' for x in (dataset_sub[i]['history'])]
for i in range(len(dataset_sub)):
    dataset_sub[i]['click'] = dataset_sub[i]['history'] + dataset_sub[i]['impressions']
    dataset_sub[i]['click'] = list(s.split('-',1) for s in dataset_sub[i]['click'])
    dataset_sub[i].pop('history', None)
    dataset_sub[i].pop('impressions', None)
output = []
for i in range(len(dataset_sub)):
    for u_click in dataset_sub[i]['click']:
        output.append({'user_id': dataset_sub[i]['user_id'], 'time': dataset_sub[i]['time'], 'news_id': u_click[0],'click':int(u_click[1])})
data = pd.DataFrame([output[0]], columns=output[0].keys())
for i in range(1,len(output)):
    data2 = pd.DataFrame([output[i]], columns=output[i].keys())
    data = pd.concat([data,data2],ignore_index=True)


In [66]:
news_id_list = list(data.loc[:,'news_id'].unique())

In [68]:
news_dataset_sub = []
for news in news_dataset:
    if news['News ID'] in news_id_list:
        news_dataset_sub.append(news)

In [70]:
len(news_dataset_sub)

12881

In [71]:
t1 = time.time()
for news in news_dataset_sub:
    url = news['URL']
    text = get_news_text(url)
    news['text'] = text
    
t2 = time.time()
print('compelte in {}'.format(t2-t1))

compelte in 4683.70077252388


In [73]:
json_data = json.dumps(news_dataset_sub)
json_file = open('base_news.json', 'w')
json_file.write(json_data)
json_file.close()

In [2]:
json_file = open('base_news.json', 'r')
data = json.load(json_file)

In [7]:
data_pd = pd.DataFrame(data)

In [28]:
index_list = data_pd[data_pd['text'] == ''].index
len(index_list)

964

In [32]:
data_pd.drop(index_list, inplace = True)

In [33]:
data_pd.reset_index(drop=True, inplace=True)

In [34]:
data_pd

,News ID,Category,SubCategory,Title,Abstract,URL,Title Entities,Abstract Entities,text
0,N93187,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId...","ZOLOTE, Ukraine — Lt. Ivan Molchanets peeked o..."
1,N75236,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi...",https://assets.msn.com/labs/mind/AACk2N6.html,[],"[{""Label"": ""National Basketball Association"", ...","Walking into the Bradley Center, I could feel ..."
2,N99744,health,medical,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re...",https://assets.msn.com/labs/mind/AAAKEkt.html,"[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI...","[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI...","[Video by Health.com]As you get older, little..."
3,N40259,news,newsworld,Chile: Three die in supermarket fire amid prot...,Three people have died in a supermarket fire a...,https://assets.msn.com/labs/mind/AAJ43pw.html,"[{""Label"": ""Chile"", ""Type"": ""G"", ""WikidataId"":...","[{""Label"": ""Santiago"", ""Type"": ""G"", ""WikidataI...",Three people have died in a supermarket fire a...
4,N22273,health,nutrition,"50 Foods You Should Never Eat, According to He...",This is so depressing.,https://assets.msn.com/labs/mind/AABDHTv.html,[],[],What's the one food you refuse to eat? Whateve...
...,...,...,...,...,...,...,...,...,...
11912,N3936,sports,basketball_nba,Raptors end Lakers' 7-game winning streak in 1...,LOS ANGELES (AP) The centerpiece of the Toro...,https://assets.msn.com/labs/mind/BBWzALP.html,"[{""Label"": ""Toronto Raptors"", ""Type"": ""O"", ""Wi...","[{""Label"": ""Toronto Raptors"", ""Type"": ""O"", ""Wi...",LOS ANGELES (AP) — The centerpiece of the Toro...
11913,N27251,news,newsworld,Evo Morales of Bolivia Accepts Asylum in Mexico,"Evo Morales, the former president of Bolivia w...",https://assets.msn.com/labs/mind/BBWzAVm.html,"[{""Label"": ""Evo Morales"", ""Type"": ""P"", ""Wikida...","[{""Label"": ""Evo Morales"", ""Type"": ""P"", ""Wikida...","LA PAZ, Bolivia — Evo Morales, the former pres..."
11914,N109606,news,newsus,"Still teaching at 95, Jimmy Carter draws devot...","PLAINS, Ga. (AP) The pilgrims arrive early a...",https://assets.msn.com/labs/mind/BBWzBrF.html,"[{""Label"": ""Jimmy Carter"", ""Type"": ""P"", ""Wikid...","[{""Label"": ""Jimmy Carter"", ""Type"": ""P"", ""Wikid...","PLAINS, Ga. (AP) — The pilgrims arrive early a..."
11915,N6982,news,newscrime,California cops respond to 2 homeless men shot...,Two homeless men living at an encampment near ...,https://assets.msn.com/labs/mind/BBWzCZq.html,"[{""Label"": ""California"", ""Type"": ""G"", ""Wikidat...","[{""Label"": ""San Francisco"", ""Type"": ""G"", ""Wiki...",Two homeless men living at an encampment near ...


In [157]:
title_dict = {}
for i in range(len(data_pd)):
    news_id = data_pd.loc[i,'News ID']
    text = data_pd.loc[i,'Title'].lower()
    title_dict[news_id] = text

In [ ]:
title_dict

In [162]:
json_data = json.dumps(title_dict)
json_file = open('news_id_title.json', 'w')
json_file.write(json_data)
json_file.close()